# Model Training

## 0. Setup

In [ ]:
# Automatic reloading
%load_ext autoreload
%autoreload 2

In [ ]:
####################
# Required Modules #
####################

# Generic/Built-in
import random
import sys 
import os

# Libs
import numpy as np
import torch
from torch.utils.data import DataLoader, random_split
import torch.optim as optim

In [ ]:
# Get the project directory 
current_dir = os.path.abspath('') # Current '\notebooks' directory
project_dir = os.path.abspath(os.path.join(current_dir, '..')) # Move up one level to project root directory

# Add the project directory to sys.path
sys.path.append(project_dir)

# Move up to project directory
os.chdir(project_dir)
os.getcwd()

In [ ]:
# Import custom modules
from src.dataset import *
from src.models import *
from src.train_eval import *

In [ ]:
# Seeding
SEED = 42

# To be safe, seed all modules for full reproducibility
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # If using CUDA
np.random.seed(SEED)
random.seed(SEED)

In [ ]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## 1. Prepare Dataset

In [ ]:
# File paths
TRAIN_PATH = "data/processed/train_set.csv"
VAL_PATH = "data/processed/val_set.csv"
TEST_PATH = "data/processed/test_set.csv"

# Hyperparameters
INPUT_SEQUENCE_LENGTH = 14 # Number of timesteps (days) in input sequence
DATASET_STRIDE = 1
BATCH_SIZE = 256

In [ ]:
# Create Dataset objects
train_dataset = CryptoDataset(
    csv_file=TRAIN_PATH,
    seq_length=INPUT_SEQUENCE_LENGTH,
    stride=DATASET_STRIDE
)
val_dataset = CryptoDataset(
    csv_file=VAL_PATH,
    seq_length=INPUT_SEQUENCE_LENGTH,
    stride=DATASET_STRIDE
)
test_dataset = CryptoDataset(
    csv_file=TEST_PATH,
    seq_length=INPUT_SEQUENCE_LENGTH,
    stride=DATASET_STRIDE
)

print("Total number of samples (sequences)")
print("Training:", len(train_dataset))
print("Validation:", len(val_dataset))
print("Test:", len(test_dataset))

In [ ]:
# Create respective data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

## 2. Training
`train_model` will save model parameters to `saved_models` directory (by default).

In [ ]:
# Hyperparameters
LEARNING_RATE = 0.001 # standard for Adam
NUM_EPOCHS = 20
SAVE_INTERVAL = 5

In [ ]:
model = CryptoInformer() # Using default settings
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
print(model)

In [ ]:
# Optional: Load previously trained model parameters (.pth file)
pretrained_model_path = "saved_models/CryptoTransformer_2025-04-09_21-31-23/CryptoTransformer_BEST_R2.pth"
# model.load_state_dict(torch.load(pretrained_model_path))

In [ ]:
training_loss_history, validation_loss_history, mae_history, r2_history, normalizer = train_model(
    model, optimizer, train_loader, val_loader,
    num_epochs=NUM_EPOCHS, save_interval=SAVE_INTERVAL,
    device=device
)

In [ ]:
save_training_plots_and_metric_history(
    training_loss_history, validation_loss_history, mae_history, r2_history, type(model).__name__
)

## 3. Test

In [ ]:
final_evaluation_loss, final_mae, final_r2 = evaluate_crypto_model(model, test_loader, normalizer)
print(f"Loss: {final_evaluation_loss:.4f}, MAE: {final_mae:.4f}, R2: {final_r2:.4f}")